## 1.1 Import Python Libraries

In [34]:
import os, sagemaker, subprocess, boto3
from sagemaker import s3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer

## 1.2 Install Ultralytics for YOLOv8 model

There are following models to choose from:
1. Object Detection:
    - yolov8l.pt
    - yolov8m.pt
    - yolov8n.pt
    - yolov8s.pt
    - yolov8x.pt
    - yolov8x6.pt
2. Segmentation:
    - yolov8l-seg.pt
    - yolov8m-seg.pt
    - yolov8n-seg.pt
    - yolov8s-seg.pt
    - yolov8x-seg.pt
3. Classification:
    - yolov8l-cls.pt
    - yolov8m-cls.pt
    - yolov8n-cls.pt
    - yolov8s-cls.pt
    - yolov8x-cls.pt

For further information, check the link: https://docs.ultralytics.com

In [35]:
!pip3 install ultralytics
from ultralytics import YOLO

## Choose a model:
model_name = 'yolov8l.pt'

YOLO(model_name)
os.system(f'mv {model_name} code/.')

bashCommand = "tar -cpzf  model.tar.gz code/"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


  0%|          | 0.00/83.7M [00:00<?, ?B/s]

## Zip the code and model into `model.tar.gz` and upload it to specific S3 bucket
Here permission is granted to the S3 bucket created with CDK and not any other bucket

In [36]:
s3_client = boto3.client('s3')
response = s3_client.list_buckets()
for bucket in response['Buckets']:
    if 'yolov8' in bucket["Name"]:
        bucket = 's3://' + bucket["Name"]

print(f'Bucket: {bucket}')
sess = sagemaker.Session(default_bucket=bucket.split('s3://')[-1])

prefix = "yolov8/demo-custom-endpoint"

Bucket: s3://yolov8sagemakerstack-yolov8s3acef217c-ev4ej8tqp4vd


In [37]:
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = get_execution_role()
print(f'Role: {role}')

model_data = s3.S3Uploader.upload("model.tar.gz", bucket + "/" + prefix)
print(f'Model Data: {model_data}')

Role: arn:aws:iam::348657513258:role/YOLOv8SageMakerStack-yolov8notebookAccessRoleB15F5-H2Y1GVVPI7L4
Model Data: s3://yolov8sagemakerstack-yolov8s3acef217c-ev4ej8tqp4vd/yolov8/demo-custom-endpoint/model.tar.gz


In [38]:
model = PyTorchModel(entry_point='inference.py',
                     model_data=model_data, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     sagemaker_session=sess)

## Deploy the model on SageMaker Endpoint:

In [39]:
INSTANCE_TYPE = 'ml.m5.4xlarge'
predictor = model.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         deserializer=JSONDeserializer())

-----!